<a href="https://colab.research.google.com/github/yungGenos/cronometro/blob/main/atividade_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#-----------------------
# RA 421110705 - kaique vinicius souza genonadio da silva
# RA 

In [15]:
# Pacotes necessários
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

# import xgboost

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import AdaBoostRegressor
from lightgbm import LGBMRegressor

# Medidas de avaliação
from sklearn import metrics

In [65]:
df_taxi_ny_train['pickup_datetime'] = pd.to_datetime(df_taxi_ny_train.pickup_datetime) # Define o tipo de dado para datetime

df_taxi_ny_train['pickup_minutes'] = df_taxi_ny_train['pickup_datetime'].dt.minute # Minutos
df_taxi_ny_train['pickup_hour'] = df_taxi_ny_train['pickup_datetime'].dt.hour # hora
df_taxi_ny_train['pickup_day'] = df_taxi_ny_train['pickup_datetime'].dt.day # dia 
df_taxi_ny_train['pickup_weekday'] = df_taxi_ny_train['pickup_datetime'].dt.weekday # dia da semana
df_taxi_ny_train['pickup_month'] = df_taxi_ny_train['pickup_datetime'].dt.month # Mês
df_taxi_ny_train['pickup_year'] = df_taxi_ny_train['pickup_datetime'].dt.year # ano
df_taxi_ny_train.head(5)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_minutes,pickup_hour,pickup_day,pickup_weekday,pickup_month,pickup_year
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,24,17,14,0,3,2016
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,43,0,12,6,6,2016
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,35,11,19,1,1,2016
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,32,19,6,2,4,2016
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,30,13,26,5,3,2016


In [66]:
# Transformando a coluna store_and_fwd_flag em binária 1 ou 0
df_taxi_ny_train['store_and_fwd_flag'] = df_taxi_ny_train['store_and_fwd_flag'].astype('category')
df_taxi_ny_train['store_and_fwd_flag'] = df_taxi_ny_train['store_and_fwd_flag'].cat.codes
df_taxi_ny_train.head(5)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_minutes,pickup_hour,pickup_day,pickup_weekday,pickup_month,pickup_year
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,455,24,17,14,0,3,2016
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,663,43,0,12,6,6,2016
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,2124,35,11,19,1,1,2016
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,0,429,32,19,6,2,4,2016
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,0,435,30,13,26,5,3,2016


In [21]:
# Carregando os dados de treino
df_taxi_ny_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/taxi_ny_train.csv')
print(df_taxi_ny_train.shape)
df_taxi_ny_train.head(5)

(1458644, 11)


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [49]:
df_taxi_ny_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


In [50]:
# Listando as colunas do arquivo de dados
df_taxi_ny_train.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')

In [7]:
# Conexão com o drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Carrega a biblioteca pandas e atribui um alias chamado "pd"
import pandas as pd

In [10]:
# Deixar colunas e linhas aparentes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Faz a leitura do arquivo train.csv 
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/taxi_ny_train.csv')

In [17]:
#1Pesquise o significado dos hiperparâmetros tanto para o Random Forest quanto para o XGBOOST

#Os hiperparâmetros (hyperparameters) são ajustes que podem ser definidos antes do treinamento do modelo e que têm um impacto significativo no desempenho do modelo. O Random Forest e o XGBoost são algoritmos de aprendizado de máquina que possuem hiperparâmetros diferentes. A seguir, vou explicar os hiperparâmetros mais comuns para cada um deles:

#Hiperparâmetros do Random Forest:
#n_estimators: O número de árvores de decisão no modelo. Valores maiores geralmente melhoram o desempenho, mas aumentam o tempo de treinamento e o uso de memória.
#min_samples_split: O número mínimo de amostras necessárias para dividir um nó interno. Valores menores podem levar a uma árvore mais complexa e overfitting, enquanto valores maiores podem levar a uma árvore menos flexível.
#min_samples_leaf: O número mínimo de amostras necessárias em uma folha. Valores menores podem levar a overfitting, enquanto valores maiores podem levar a uma árvore menos flexível.
#max_features: O número máximo de recursos que cada árvore é permitida a considerar para fazer uma divisão. Valores menores podem reduzir a variância, mas podem aumentar o viés.

#Hiperparâmetros do XGBoost:
#learning_rate: O tamanho do passo de atualização em cada iteração enquanto se movimenta em direção ao mínimo da função de perda. Valores menores reduzem o viés, mas aumentam o tempo de treinamento.
#max_depth: A profundidade máxima de cada árvore de decisão. Valores maiores geralmente aumentam o desempenho, mas também podem aumentar o risco de overfitting.
#min_child_weight: O número mínimo de amostras necessárias em cada nó. Valores maiores podem levar a uma árvore mais conservadora, enquanto valores menores podem levar a overfitting.
#subsample: A fração de amostras a serem usadas para cada árvore de decisão. Valores menores podem reduzir a variância, mas podem aumentar o viés.
#colsample_bytree: A fração de recursos a serem usados para cada árvore de decisão. Valores menores podem reduzir a variância, mas podem aumentar o viés.
#Estes são apenas alguns dos hiperparâmetros disponíveis para cada modelo. É importante destacar que a escolha de valores apropriados para esses hiperparâmetros pode ser um processo complexo e envolve experimentação e ajuste fino.

In [19]:
#2 Justifique a escolha dos hiperparâmetros com a explicação 

#Ao usar o hiperparâmetro random forest, é possível escolher um número aleatório de árvores dentro de um determinado intervalo. Esse método pode ajudar a encontrar um número ótimo de árvores que equilibre a precisão do modelo com sua eficiência. 
#No entanto, temos  em mente que a escolha do número ideal de árvores pode variar dependendo do conjunto de dados e do problema específico que estamos tentando resolver. Portanto, 
#é sempre importante experfrom sklearn.ensemble import RandomForestClassifie



In [107]:
#3 Altere o percentual de treino e teste

# Separando as informações em y e x
y = df_taxi_ny_train['trip_duration']
X = df_taxi_ny_train.drop('trip_duration', axis=1)

# Separando as informações do arquivo de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Leitura do arquivo CSV com pandas
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/taxi_ny_train.csv')

# Separando as features das labels
X = df.drop('trip_duration', axis=1)
y = df['trip_duration']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Verificando as dimensões dos dados de treino e teste
print("Dimensões do conjunto de treino:", X_train.shape, y_train.shape)
print("Dimensões do conjunto de teste:", X_test.shape, y_test.shape)



Dimensões do conjunto de treino: (1021050, 10) (1021050,)
Dimensões do conjunto de teste: (437594, 10) (437594,)


In [34]:

pip install scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
# Listando as colunas do arquivo de dados
df_taxi_ny_train.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')

In [39]:
# Separando as informações em y e x
y = df_taxi_ny_train['trip_duration']
X = df_taxi_ny_train.drop('trip_duration', axis=1)

# Separando as informações do arquivo de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

In [43]:
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5, n_jobs=-1, error_score='raise')


In [52]:
# Frequencia da coluna store_and_fwd_flag depois da alteração anterior
df_taxi_ny_train['store_and_fwd_flag'].value_counts(dropna=False)

N    1450599
Y       8045
Name: store_and_fwd_flag, dtype: int64

In [57]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, max_depth=10, max_features='sqrt', random_state=42)


In [64]:
# Frequencia da coluna store_and_fwd_flag
df_taxi_ny_train['store_and_fwd_flag'].value_counts(dropna=False)

N    1450599
Y       8045
Name: store_and_fwd_flag, dtype: int64

In [69]:
# Separando as informações em y e x
y = df_taxi_ny_train['trip_duration']
X = df_taxi_ny_train.drop('trip_duration', axis=1)

# Separando as informações do arquivo de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

In [74]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Leitura do arquivo CSV com pandas
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/taxi_ny_train.csv')

# Separando as features das labels
X = df.drop('trip_duration', axis=1)
y = df['trip_duration']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criando o objeto da regressão Random Forest
rf_regressor = RandomForestRegressor(random_state=42)





In [79]:
# Frequencia da coluna store_and_fwd_flag
df_taxi_ny_train['store_and_fwd_flag'].value_counts(dropna=False)

0    1450599
1       8045
Name: store_and_fwd_flag, dtype: int64

In [100]:
# Listando as colunas do arquivo de dados
df_taxi_ny_train.columns


Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'pickup_minutes', 'pickup_hour', 'pickup_day',
       'pickup_weekday', 'pickup_month', 'pickup_year'],
      dtype='object')

In [103]:
# Deleta as colunas que não iremos utilizar

# O argumento axis especifica se as linhas (0) ou colunas (1) devem ser descartadas.
# O argumento inplace especifica a remoção das colunas sem reatribuir o DataFrame. (persiste em memoria)

cols = ['id','dropoff_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'pickup_datetime']
df_taxi_ny_train.drop(cols, axis=1, inplace=True)
print(df_taxi_ny_train.shape)
df_taxi_ny_train.head(5)

(1458644, 10)


,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,pickup_minutes,pickup_hour,pickup_day,pickup_weekday,pickup_month,pickup_year
0,2,1,0,455,24,17,14,0,3,2016
1,1,1,0,663,43,0,12,6,6,2016
2,2,1,0,2124,35,11,19,1,1,2016
3,2,1,0,429,32,19,6,2,4,2016
4,2,1,0,435,30,13,26,5,3,2016


In [105]:
# Frequencia da coluna store_and_fwd_flag depois da alteração anterior
df_taxi_ny_train['store_and_fwd_flag'].value_counts(dropna=False)

0    1450599
1       8045
Name: store_and_fwd_flag, dtype: int64

In [109]:
# Criando o objeto da regressão Random Forest
rf_regressor = RandomForestRegressor(random_state=42)

rf_regressor.fit(x_train, y_train)


NameError: ignored

In [75]:

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

# Carregando a tabela em um dataframe pandas
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/taxi_ny_train.csv')

# Separando as features das labels
X = df.drop('trip_duration', axis=1)
y = df['trip_duration']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definindo os valores dos hiperparâmetros a serem testados
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2', 0.5]
}

# Criando o objeto GridSearchCV para encontrar os melhores hiperparâmetros
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5, n_jobs=-1)

# Treinando o modelo com os melhores hiperparâmetros encontrados pelo GridSearchCV
# Métricas de avaliação

# Erro Absoluto Médio (MAE): mede as distâncias entre valores preditos e observados. 
# Essa métrica não é muito influenciada pelos outliers. Quanto menor seu valor, melhor
MAE_rf = metrics.mean_absolute_error(y_test, rf_result)
print('MAE_rf:', metrics.mean_absolute_error(y_test, rf_result))

# Erro Quadrático Médio (MSE): indicada para problemas nos quais grandes erros não são tolerados. 
# Mais difícil de interpretar, pois eleva a unidade ao quadrado. Quanto menor seu valor, melhor
MSE_rf = metrics.mean_squared_error(y_test, rf_result)
print('MSE_rf:', metrics.mean_squared_error(y_test, rf_result))

# Raiz do Erro Quadrático Médio (RMSE): tem a mesma unidade dos valores observados e preditos. 
# Seu resultado é uma medida do desvio médio (erro) entre observado e predito
RMSE_rf = np.sqrt(metrics.mean_squared_error(y_test, rf_result))
print('RMSE_rf:', np.sqrt(metrics.mean_squared_error(y_test, rf_result)))

# R2: calcula qual a porcentagem da variância que pode ser explicada pelo modelo de regressão. 
# Informa o quão “próximo” as medidas reais estão do nosso modelo. Pode variar entre 0 e 1
R2_SCORE_rf = metrics.r2_score(y_test, rf_result)

# Imprimindo o melhor conjunto de hiperparâmetros e a acurácia do modelo treinado
print("Melhores hiperparâmetros:", grid_search.best_params_)
print("Acurácia do modelo:", grid_search.best_score_)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


ValueError: ignored